In [ ]:
import os, json, random, numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import csv

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_sst2_and_tokenizer(tokenizer_name="bert-base-uncased", max_len=64):
    raw = load_dataset("glue", "sst2")
    tok = AutoTokenizer.from_pretrained(tokenizer_name)
    if tok.pad_token is None:
        tok.add_special_tokens({"pad_token": "[PAD]"})
    pad_id = tok.pad_token_id
    vocab_size = tok.vocab_size

    def tok_fn(batch):
        out = tok(batch["sentence"], padding="max_length", truncation=True, max_length=max_len, return_tensors=None)
        attn = out["attention_mask"]
        lengths = [int(sum(m)) for m in attn]
        out2 = {"input_ids": out["input_ids"], "attention_mask": attn, "length": lengths}
        if "label" in batch: out2["label"] = batch["label"]
        return out2

    tokenized = {}
    for split in raw.keys():
        tokenized[split] = raw[split].map(tok_fn, batched=True, remove_columns=raw[split].column_names)
        tokenized[split].set_format(type="torch")
    return tokenized, tok, vocab_size, pad_id

In [ ]:
class SST2TorchDataset(Dataset):
    def __init__(self, ds, is_test=False):
        self.ds = ds; self.is_test = is_test
    def __len__(self): return len(self.ds)
    def __getitem__(self, idx):
        ex = self.ds[idx]
        x = ex["input_ids"]
        length = ex["length"]
        y = -1 if self.is_test else int(ex["label"])
        return x, length, y

In [ ]:
def collate_fn(batch):
    xs, lengths, ys = zip(*batch)
    xs = torch.stack(xs, dim=0)
    lengths = torch.tensor(lengths, dtype=torch.long)
    ys = torch.tensor(ys, dtype=torch.long)
    lengths, perm = lengths.sort(descending=True)
    xs = xs[perm]; ys = ys[perm]
    return xs, lengths, ys

In [ ]:
class BasicRNNSentiment(nn.Module):
    def __init__(self, vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=0, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.RNN(emb_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x, lengths):
        emb = self.embedding(x)
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=True)
        _, h_n = self.rnn(packed)
        out = self.dropout(h_n[-1])
        return self.fc(out)

In [ ]:
class GRUSentiment(nn.Module):
    def __init__(self, vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=0, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.gru = nn.GRU(emb_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x, lengths):
        emb = self.embedding(x)
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=True)
        _, h_n = self.gru(packed)
        out = self.dropout(h_n[-1])
        return self.fc(out)

In [ ]:
class LSTMSentiment(nn.Module):
    def __init__(self, vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=0, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x, lengths):
        emb = self.embedding(x)
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=True)
        _, (h_n, _) = self.lstm(packed)
        out = self.dropout(h_n[-1])
        return self.fc(out)

In [ ]:
class BiLSTMSentiment(nn.Module):
    def __init__(self, vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, bidirectional=True, dropout=0.3, pad_idx=0, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
    def forward(self, x, lengths):
        emb = self.embedding(x)
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=True)
        _, (h_n, _) = self.lstm(packed)
        h_cat = torch.cat((h_n[-2], h_n[-1]), dim=1)
        out = self.dropout(h_cat)
        return self.fc(out)

In [ ]:
def train_and_eval_one_model(model, name, train_dl, dev_dl, device, epochs=20, lr=1e-3, grad_clip=1.0):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_f1 = 0.0
    best_path = f"{name}_best.pt"
    history = {'epoch': [], 'train_loss': [], 'train_acc': [], 'dev_loss': [], 'dev_acc': [], 'dev_f1': []}

    for ep in range(1, epochs + 1):
        model.train()
        tot_loss = 0
        preds_all, labels_all = [], []
        for x, lengths, y in train_dl:
            x, lengths, y = x.to(device), lengths.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x, lengths)
            loss = criterion(logits, y)
            loss.backward()
            if grad_clip is not None:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()

            tot_loss += loss.item() * x.size(0)
            preds_all.extend(logits.argmax(1).detach().cpu().tolist())
            labels_all.extend(y.detach().cpu().tolist())

        train_loss = tot_loss / len(train_dl.dataset)
        train_acc = accuracy_score(labels_all, preds_all)
        _, _, train_f1, _ = precision_recall_fscore_support(labels_all, preds_all, average="binary", zero_division=0)

        model.eval()
        tot_loss = 0
        preds_all, labels_all = [], []
        with torch.no_grad():
            for x, lengths, y in dev_dl:
                x, lengths, y = x.to(device), lengths.to(device), y.to(device)
                logits = model(x, lengths)
                loss = criterion(logits, y)
                tot_loss += loss.item() * x.size(0)
                preds_all.extend(logits.argmax(1).detach().cpu().tolist())
                labels_all.extend(y.detach().cpu().tolist())

        dev_loss = tot_loss / len(dev_dl.dataset)
        dev_acc = accuracy_score(labels_all, preds_all)
        p, r, dev_f1, _ = precision_recall_fscore_support(labels_all, preds_all, average="binary", zero_division=0)
        cm = confusion_matrix(labels_all, preds_all)

        history['epoch'].append(ep)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['dev_loss'].append(dev_loss)
        history['dev_acc'].append(dev_acc)
        history['dev_f1'].append(dev_f1)

        if dev_f1 > best_f1:
            best_f1 = dev_f1
            torch.save(model.state_dict(), best_path)

        print(f"[{name}] [Epoch {ep}] Train loss={train_loss:.4f} acc={train_acc:.4f} | Dev loss={dev_loss:.4f} acc={dev_acc:.4f} f1={dev_f1:.4f}")

    model.load_state_dict(torch.load(best_path, map_location=device))
    model.eval()
    preds_all, labels_all = [], []
    tot_loss = 0
    with torch.no_grad():
        for x, lengths, y in dev_dl:
            x, lengths, y = x.to(device), lengths.to(device), y.to(device)
            logits = model(x, lengths)
            loss = criterion(logits, y)
            tot_loss += loss.item() * x.size(0)
            preds_all.extend(logits.argmax(1).detach().cpu().tolist())
            labels_all.extend(y.detach().cpu().tolist())
    final_loss = tot_loss / len(dev_dl.dataset)
    final_acc = accuracy_score(labels_all, preds_all)
    p, r, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average="binary", zero_division=0)
    final_cm = confusion_matrix(labels_all, preds_all)

    summary = {"dev_loss": final_loss, "dev_acc": final_acc, "precision": p, "recall": r, "f1": f1, "cm": final_cm.tolist()}
    return history, summary, best_path

In [ ]:
# --------------------- Plotting ---------------------
def plot_history(history_dicts, out_dir="figs"):
    """
    history_dicts: dict(name -> history dict)
    Creates:
      - fig1: accuracy (all models on one chart)
      - fig2: loss (all models on one chart)
    """
    os.makedirs(out_dir, exist_ok=True)
    # Figure 1: Accuracy
    plt.figure(figsize=(9,6))
    for name, h in history_dicts.items():
        plt.plot(h['epoch'], h['train_acc'], label=f"{name} Train")
        plt.plot(h['epoch'], h['dev_acc'],   label=f"{name} Val")
    plt.title("Figure 1: Training vs Validation Accuracy (All Models)")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.grid(True)
    f1_path = os.path.join(out_dir, "fig1_accuracy_all_models.png")
    plt.savefig(f1_path, dpi=150, bbox_inches="tight"); plt.close()

    # Figure 2: Loss
    plt.figure(figsize=(9,6))
    for name, h in history_dicts.items():
        plt.plot(h['epoch'], h['train_loss'], label=f"{name} Train")
        plt.plot(h['epoch'], h['dev_loss'],   label=f"{name} Val")
    plt.title("Figure 2: Training vs Validation Loss (All Models)")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.grid(True)
    f2_path = os.path.join(out_dir, "fig2_loss_all_models.png")
    plt.savefig(f2_path, dpi=150, bbox_inches="tight"); plt.close()
    return f1_path, f2_path

In [ ]:
def plot_f1_summary(summaries, out_dir="figs"):
    os.makedirs(out_dir, exist_ok=True)
    names = list(summaries.keys())
    f1s = [summaries[n]["f1"] for n in names]
    plt.figure()
    plt.bar(names, f1s)
    for i, v in enumerate(f1s):
        plt.text(i, v + 0.01, f"{v:.3f}", ha="center")
    plt.ylim(0, max(f1s) + 0.1)
    plt.title("Best Validation F1 by Architecture")
    plt.ylabel("F1-score"); plt.grid(axis="y", linestyle="--", alpha=0.6)
    out_path = os.path.join(out_dir, "summary_f1.png")
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()
    return out_path

In [ ]:
def plot_confusion_matrix(cm, name, out_dir="figs"):
    os.makedirs(out_dir, exist_ok=True)
    fig, ax = plt.subplots(figsize=(4.8,4))
    im = ax.imshow(cm)
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(['Negative','Positive']); ax.set_yticklabels(['Negative','Positive'])
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(f"Figure 3: Confusion Matrix — {name}")
    for i in range(2):
        for j in range(2):
            ax.text(j, i, int(cm[i, j]), ha="center", va="center")
    fig.tight_layout()
    out = os.path.join(out_dir, f"fig3_cm_{name.lower()}.png")
    plt.savefig(out, dpi=150, bbox_inches="tight")
    plt.close()
    return out

In [ ]:
# --------------------- Main ---------------------
def main():
    MAX_LEN = 64
    tokenized, tokenizer, vocab_size, pad_id = load_sst2_and_tokenizer(max_len=MAX_LEN)

    train_ds = SST2TorchDataset(tokenized["train"], is_test=False)
    dev_ds   = SST2TorchDataset(tokenized["validation"], is_test=False)
    BATCH_SIZE = 64
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
    dev_dl   = DataLoader(dev_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    models = {
        "BasicRNN": BasicRNNSentiment(vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=pad_id, num_classes=2),
        "GRU":      GRUSentiment(   vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=pad_id, num_classes=2),
        "LSTM":     LSTMSentiment(  vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, dropout=0.3, pad_idx=pad_id, num_classes=2),
        "BiLSTM":   BiLSTMSentiment(vocab_size, emb_dim=100, hidden_dim=128, num_layers=1, bidirectional=True, dropout=0.3, pad_idx=pad_id, num_classes=2),
    }

    histories = {}
    summaries = {}
    checkpoints = {}
    os.makedirs("figs", exist_ok=True)
    os.makedirs("outputs", exist_ok=True)

    for name, mdl in models.items():
        hist, summ, ckpt = train_and_eval_one_model(
            model=mdl,
            name=name,
            train_dl=train_dl,
            dev_dl=dev_dl,
            device=DEVICE,
            epochs=20,
            lr=1e-3,
            grad_clip=1.0
        )
        histories[name] = hist
        summaries[name] = summ
        checkpoints[name] = ckpt

        # per-model confusion matrix fig (Figure 3a–d)
        import numpy as np
        plot_confusion_matrix(np.array(summ["cm"]), name)

    # Figure 1 & 2 (all models together)
    plot_history(histories)

    # Figure 4 (F1 summary bar)
    plot_f1_summary(summaries)

    # Save CSV summary + histories
    csv_path = os.path.join("outputs", "model_summary.csv")
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "Dev Acc", "Precision", "Recall", "F1", "Dev Loss", "ConfusionMatrix"])
        for name, s in summaries.items():
            writer.writerow([name, f"{s['dev_acc']:.4f}", f"{s['precision']:.4f}",
                             f"{s['recall']:.4f}", f"{s['f1']:.4f}", f"{s['dev_loss']:.4f}",
                             json.dumps(s['cm'])])

    with open(os.path.join("outputs", "histories.json"), "w") as f:
        json.dump(histories, f)

    print("\n=== Best Dev Metrics ===")
    for name, s in summaries.items():
        print(f"{name:8s} | Acc={s['dev_acc']:.4f}  P={s['precision']:.4f}  R={s['recall']:.4f}  F1={s['f1']:.4f}  | CM={s['cm']}")
    print(f"Summary CSV: {csv_path}")
    print("Figures saved in ./figs")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

[BasicRNN] [Epoch 1] Train loss=0.5268 acc=0.7301 | Dev loss=0.5268 acc=0.7557 f1=0.7609
[BasicRNN] [Epoch 2] Train loss=0.3154 acc=0.8735 | Dev loss=0.5500 acc=0.7557 f1=0.7599
[BasicRNN] [Epoch 3] Train loss=0.2364 acc=0.9117 | Dev loss=0.5468 acc=0.7913 f1=0.7869
[BasicRNN] [Epoch 4] Train loss=0.1932 acc=0.9313 | Dev loss=0.6281 acc=0.7959 f1=0.8053
[BasicRNN] [Epoch 5] Train loss=0.1642 acc=0.9420 | Dev loss=0.6730 acc=0.7890 f1=0.7946
[BasicRNN] [Epoch 6] Train loss=0.1422 acc=0.9502 | Dev loss=0.6745 acc=0.7856 f1=0.7925
[BasicRNN] [Epoch 7] Train loss=0.1255 acc=0.9561 | Dev loss=0.6988 acc=0.7878 f1=0.8013
[BasicRNN] [Epoch 8] Train loss=0.1072 acc=0.9633 | Dev loss=0.7291 acc=0.7993 f1=0.8049
[BasicRNN] [Epoch 9] Train loss=0.0946 acc=0.9671 | Dev loss=0.7959 acc=0.7970 f1=0.8053
[BasicRNN] [Epoch 10] Train loss=0.0840 acc=0.9707 | Dev loss=0.8554 acc=0.8028 f1=0.8041
[BasicRNN] [Epoch 11] Train loss=0.0758 acc=0.9739 | Dev loss=0.9434 acc=0.8028 f1=0.8063
[BasicRNN] [Epoch 1